<a href="https://colab.research.google.com/github/plediii/rnn-quantum-circuits/blob/master/Unitary_Time_Evolution_Operators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

Here I'm going to construct variable unitary matrices parametrized from real matrices, and then optimize the the variable matrices to produce desired unitary matrices.

In [47]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  print('Failed to upgrade tensorflow')
  pass

import tensorflow as tf

import numpy as np

print(tf.__version__)

2.0.0


# Recall quantum state vectors

The most important property of a unitary matrix is that it conserves the norm of our quantum state vectors. 

So first let's recall how to to calculate the norm of our vector, and construct a normalized quantum state vector so we can verify the unitarity of our constructed matrices.

We say that the adjoint of `x` is `x*`. The square of `x` is `x* x`, and the norm of `x` is the square root of its square.

These definitions can be expressed in tensorflow in the following way:

In [0]:
def dot(a, b):
  """a* b"""
  return tf.matmul(tf.linalg.adjoint(a), b)[0][0]

def norm(x):
  """The dot x with itself, and take the square root"""
  return tf.sqrt(dot(x, x))

def normalized(x):
  return x / norm(x)

Now, a quantum state vector is any normalized complex vector.

A qbit is a 2-dimensional quantum vector.

In [4]:
def random_quantum_state(n):
  """Construct a random n-dimensonal quantum state vector"""
  return normalized(tf.complex(tf.random.normal((n, 1)), tf.random.normal((n, 1))))

qbit = random_quantum_state(2)
print('qbit norm = ', tf.math.real(norm(qbit)).numpy())
print('qbit = ', qbit.numpy())

qbit norm =  1.0
qbit =  [[ 0.69474643-0.23535512j]
 [-0.6457317 +0.21205197j]]


# What is a unitary matrix?

Generally, when you mulitply an input vector by a matrix, the resulting output vector may have a different norm than the input vector.

Now, suppose we want to represent time evolution of our quantum state vector with multiplication by a particular matrix. Both the input quantum state, and the output quantum state must have the same norm, `1`. 

Let's assume an initial state `x`, a next state in time `x'`, and a time evolution matrix `U`:
```
  x' = Ux
```
If both `x` and `x'` are quantum states, then we require their norms and squares to be `1`.
```
      (x')* x' == 1 
  and  x* x    == 1
 ```

This requirement constrains the possible matrices `U`. Since
```
  (x')* x = (Ux)* Ux 
          = x* U* U x 
          = x* x
```
In order for this to be true, we must have that `U* U = 1`, the identiy matrix. In other words `U*` is the inverse of `U`.

How can we construct general unitary matrices? Let's start with Hermitian matrices. 

# Constructing a Hermitian symmetric matrix

Unitary matrices are very closely related to symmetric matrices. 

For a unitary matrix `U`, `U* U = 1`. 

For Hermitian symmetric matrices, `H* = H`.

Given an arbitrary matrix `M`, it's easy to construct a Hermitian matrix by adding `M` to its adjoint.

Then `(M* + M)* = M + M* = M* + M`.



In [5]:
M = tf.complex(tf.random.uniform((2, 2)), tf.random.uniform((2, 2)))
M

<tf.Tensor: id=53, shape=(2, 2), dtype=complex64, numpy=
array([[0.77460563+0.68357384j, 0.69898   +0.48072052j],
       [0.20853305+0.4314425j , 0.07056069+0.5087286j ]], dtype=complex64)>

In [6]:
H = M + tf.linalg.adjoint(M)
H

<tf.Tensor: id=56, shape=(2, 2), dtype=complex64, numpy=
array([[1.5492113 +0.j        , 0.907513  +0.04927802j],
       [0.907513  -0.04927802j, 0.14112139+0.j        ]], dtype=complex64)>

In [7]:
tf.reduce_all(tf.linalg.adjoint(H) == H).numpy()

True

# Constructing a Unitary Matrix

Unitary matrices have an interesting relationship with Hermitian matrices.
It turns out that any unitary matrix can be expressed as an exponential of a Hermitian matrix!

That is, we can define an analog of exponentiation operation, which, when applied to matrices satisfies the following identities:
* `exp{M*} = (exp{M})*`
* `exp{-M}exp{M} = 1`.

Then, given a Hermitian matrix `H`, we can create a unitary matrix as:
```
U = exp{iH}.
```
Then, we can verify the unitarity of our constructed matrix:
```
U* U = (exp{iH})* exp{iH}
     = exp{(iH)*} exp{iH}
     = exp{-iH} exp{iH} 
     = 1.
```

What's more, Tensorflow provides a fast function for computing matrix exponentials.

This approximation yields a numerically approximate unitary matrix, conserving the norm of our qbit states.

In [0]:
U = tf.linalg.expm(tf.complex(0., 1.) * H)

In [9]:
tf.math.real(norm(tf.matmul(U, qbit))).numpy()

1.0

In [10]:
U.numpy()

array([[-0.14680567+0.67670584j, -0.5648827 +0.44880378j],
       [-0.5129644 +0.507331j  ,  0.68938375-0.06505959j]],
      dtype=complex64)

What's more, we can definite a matrix logarithm, which inverts our matrix exponential, allowing us to move back and forth between the space of unitary and Hermitian variables. 

# Constructing a variable unitary matrix using minimal parameters

In tensorflow, it's a effortless to construct a variable matrix, and, using the above path, we can transform a variable matrix into a variable unitary matrix. 

However, constructing a Hermitian matrix by adding a matrix to its adjoint is a bit wasteful in terms of the count of real parameters involved. Later, we'll want to compute derivatives against these parameters, so the fewer parameters we have, the less memory and time we'll need to optimize our functions.

Creating a Hermitian matrix by adding a matrix to its adjoint requires `2xNxN` random numbers, two for each complex number in the original matrix. However, it turns out that only `NxN` real numbers are required.

In the `2x2` case, we initially construct `8` real numbers. We can easily count `8` distinct real numbers in the initial random complex matrix.

In [11]:
M = tf.complex(tf.random.uniform((2, 2)), tf.random.uniform((2, 2)))
M.numpy()

array([[0.37151647+0.3203796j , 0.01461518+0.8826269j ],
       [0.86200917+0.57722914j, 0.26748002+0.48068213j]], dtype=complex64)

However, close inspection shows only `4` distinct non-zero real numbers in the constructed adjoint matrix. 

In [12]:
H = (M + tf.linalg.adjoint(M))
H.numpy()

array([[0.74303293+0.j        , 0.87662435+0.30539775j],
       [0.87662435-0.30539775j, 0.53496003+0.j        ]], dtype=complex64)

The reason can be understood in that Hermitian constraint is symmetry constraint on the real part, and an anti-symmetry constraint on the imaginary part.

The symmetry constraint eliminates almost half our real numbers, since the upper and lower triangles of the real part are identical:
`Real{H}[i][j] = Real{H*}[i][j] = Real{H}[j][i]`.

It also eliminates the diagonal of the imaginary part, and makes the upper and lower triangles negatives of one another:
`Imag{H}[i][j] = Imag{H*}[i][j] = -Imag{H}[j][i]`.


In [0]:
realH = tf.math.real(H)
assert (realH[0][1] == realH[1][0]).numpy()
imagH = tf.math.imag(H)
assert (imagH[0][1] == -imagH[1][0]).numpy()
assert (imagH[0][0] == 0.).numpy()

The entire space of complex hermitian matrices can be parameterized by just `NxN` real numbers. 

That is, given an `NxN` matrix, we can use the diagonal and upper triangle for the real part, and the lower triangle for the imaginary part.

We can codify that as the following:

In [0]:
@tf.function
def hermitian_from_real(m):
  def hermitian_element(m, idx, jdx):
    if idx == jdx:
      return tf.complex(m[idx][jdx], 0.)
    elif idx < jdx:
      return tf.complex(m[idx][jdx], m[jdx][idx])
    else:
      return tf.complex(m[jdx][idx], -m[idx][jdx])
  N = m.shape[0]
  return tf.convert_to_tensor([[hermitian_element(m, idx, jdx) for idx in range(N)] for jdx in range(N)])

H = hermitian_from_real(tf.random.normal((3, 3)))
assert tf.reduce_all(H == tf.linalg.adjoint(H))

We can construct just the `iH` in the unitary `U = exp{iH}` similarly. Leaving us with a parameterization of unitary matrices from `NxN` real matrices. 

We can verify that the resulting `U* U` is quite close to `I`.

In [22]:
@tf.function
def unitary_from_real(m):
  return tf.linalg.expm(tf.complex(0., 1.) * hermitian_from_real(m))

U = unitary_from_real(tf.random.normal((3, 3)))
tf.abs(tf.complex(tf.eye(3), tf.zeros((3, 3))) - tf.matmul(tf.linalg.adjoint(U), U))

<tf.Tensor: id=1470, shape=(3, 3), dtype=float32, numpy=
array([[1.1920929e-07, 6.1439060e-08, 1.4675942e-07],
       [6.1439060e-08, 1.7881393e-07, 7.8231096e-08],
       [1.6049042e-07, 7.5339337e-08, 6.0404695e-08]], dtype=float32)>

# Learn Pauli Spin Matrices

A famous collection of unitary matrices are [the Pauli matrices](https://en.wikipedia.org/wiki/Pauli_matrices). Let's see if we can learn the Pauli matrices by optimizing an underlying hermitian matrix.

In [16]:
pauli_matrices = [
          tf.complex([[0., 1.,], [1., 0.,]], tf.zeros((2, 2))),
          tf.complex(tf.zeros((2, 2)), [[0., -1.,], [1., 0.,]]),
          tf.complex([[1., 0.,], [0., -1.]], tf.zeros((2, 2)))
]
for p  in pauli_matrices:
  print(p.numpy())

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

The following functions will find an optimal hermitian matrix to produce a desired unitary matrix.

In [0]:
def unitary_loss(m, t):
  diff = unitary_from_real(m) - t
  return tf.reduce_sum(tf.math.real(tf.math.conj(diff) * diff))

def learn_real_for_unitary(t):
  m = tf.Variable(tf.random.normal((2, 2)))
  for _ in range(1000):
    with tf.GradientTape() as tape:
      loss = unitary_loss(m, t)
    g = tape.gradient(loss, [m])
    optimizer.apply_gradients(zip(g, [m]))
  return m


Optimizing with respect to the Pauli matrices gives the following results:

In [44]:
for p in pauli_matrices:
  print('\n\nLearning Pauli matrix =\n ', p.numpy())
  m = learn_real_for_unitary(p)
  print('Final loss = ', unitary_loss(m, p).numpy())
  print('Final Hermitian = \n', hermitian_from_real(m).numpy())
  print('Final unitary = \n', unitary_from_real(m).numpy())




Learning Pauli matrix =
  [[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
Final loss =  1.4444395e-13
Final Hermitian = 
 [[-1.5707965+0.000000e+00j  1.5707963-8.737947e-08j]
 [ 1.5707963+8.737947e-08j -1.5707958+0.000000e+00j]]
Final unitary = 
 [[-1.1509718e-07-1.0458851e-07j  1.0000000e+00+1.7881393e-07j]
 [ 1.0000001e+00+1.4901161e-07j  2.2333263e-07-4.4616442e-08j]]


Learning Pauli matrix =
  [[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]
Final loss =  1.10826645e-13
Final Hermitian = 
 [[-1.5707961e+00+0.j        -2.2026612e-09-1.5707964j]
 [-2.2026612e-09+1.5707964j -1.5707964e+00+0.j       ]]
Final unitary = 
 [[ 1.6213713e-07-6.280289e-08j  1.7881393e-07-1.000000e+00j]
 [-1.4901161e-07+9.999999e-01j -1.1034668e-07-5.278430e-09j]]


Learning Pauli matrix =
  [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
Final loss =  3.974162e-13
Final Hermitian = 
 [[ 9.0306398e-24+0.0000000e+00j -9.2925724e-25-2.2543146e-23j]
 [-9.2925724e-25+2.2543146e-23j -3.1415918e+00+0.0000000e+00j]]
Final unitary = 
 [[ 1.0000000e+

We can see from the above that the Hemritian matrices corresponding to the Pauli matrices are:
```
[[ sqrt(2),   sqrt(2)],
 [ sqrt(2),  -sqrt(2)]],
```
```
[[ -sqrt(2),   -i sqrt(2)],
 [ i sqrt(2),  -sqrt(2)]],
```
and
```
[[ 0,  0],
 [ 0,  -pi]],
```
respectively.